In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/covid_19_india.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
#plt.rcParams['figure.figsize']=17,8
import cufflinks as cf
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot
import folium 
from folium import plugins
plt.rcParams['figure.figsize'] = 10, 12
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pyo.init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
df= pd.read_csv(r"../input/covid19-in-india/covid_19_india.csv")
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['ConfirmedIndianNational','ConfirmedForeignNational'],axis=1,inplace=True)

In [ ]:
df.shape

# Let's See WHats the race between Recovered and Deaths

In [ ]:
f, ax = plt.subplots(figsize=(28, 28))
data = df[['State/UnionTerritory','Confirmed','Cured','Deaths']]   
data.sort_values('Confirmed',ascending=False,inplace=True)
sns.set_color_codes("pastel")
sns.barplot(x="Confirmed", y="State/UnionTerritory", data=data,label="Total", color="red")
sns.set_color_codes("muted")
sns.barplot(x="Cured", y="State/UnionTerritory", data=data, label="Cured", color="green")
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 700000), ylabel="",xlabel="Cases")
sns.despine(left=True, bottom=True)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst = True)

## Check The Confirmed, Recovered and Death Trolls in India 

In [ ]:
df_confirmed_india=df.groupby('Date')['Confirmed'].sum().reset_index()
df_cured_india=df.groupby('Date')['Cured'].sum().reset_index()
df_death_india=df.groupby('Date')['Deaths'].sum().reset_index()

In [ ]:
df_confirmed_india

In [ ]:
print("The Confirmed Cases are",df_confirmed_india.Confirmed.max())
print("The Recovered Cases are",df_cured_india.Cured.max())
print("The Deaths Cases are",df_death_india.Deaths.max())

## Let's See the Day-wise cases on India

> ## Confirmed Day wise Cases

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_confirmed_india['Date'], y = df_confirmed_india['Confirmed'], mode='lines+markers',name='Total Cases'))
fig.update_layout(title_text='Trend of Coronavirus Cases in India (Cumulative cases)',plot_bgcolor='rgb(230, 230, 230)')
fig.show()
# New COVID-19 cases reported daily in India
import plotly.express as px
fig = px.bar(df_confirmed_india, x="Date", y="Confirmed", barmode='group', height=400)
fig.update_layout(title_text='Coronavirus Cases in India on daily basis',plot_bgcolor='gold')
fig.show()

## Deaths Day-Wise 

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_death_india['Date'], y = df_death_india['Deaths'], mode='lines+markers',name='Total Deaths',line=dict(color='Red', width=2)))
fig.update_layout(title_text='Trend of Death Cases in India',plot_bgcolor='rgb(230, 230, 230)')
fig.show()
# New COVID-19 cases reported daily in India
import plotly.express as px
fig = px.bar(df_death_india, x="Date", y="Deaths", barmode='group', height=500)
fig.update_layout(title_text='Coronavirus Deaths in India on daily basis',plot_bgcolor='lime')
fig.show()

## Let's Do in One graph to see te day-Wise Confirmed, Deaths and Cured cases

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_confirmed_india['Date'], y=df_confirmed_india['Confirmed'], mode='lines+markers', name='Confirmed',line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=df_death_india['Date'], y=df_death_india['Deaths'], mode='lines+markers', name='Deaths', line=dict(color='Red', width=2)))
fig.add_trace(go.Scatter(x=df_cured_india['Date'], y=df_cured_india['Cured'], mode='lines+markers', name='Recovered', line=dict(color='Green', width=2)))
fig.update_layout(title='India COVID-19 Cases', xaxis_tickfont_size=14,yaxis=dict(title='Number of Cases'))
fig.show()

# Let's Now Analyze State-Wise Cases and Recovery

In [ ]:
import numpy as np
state_cases = df.groupby('State/UnionTerritory')['Confirmed','Deaths','Cured'].max().reset_index()
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Deaths']+ state_cases['Cured'])
state_cases["Death Rate (per 100)"] = np.round(100*state_cases["Deaths"]/state_cases["Confirmed"],2)
state_cases["Cure Rate (per 100)"] = np.round(100*state_cases["Cured"]/state_cases["Confirmed"],2)

In [ ]:
state_cases

In [ ]:
state_cases.head()

In [ ]:
state_cases['State/UnionTerritory'].unique()

In [ ]:
state_cases['State/UnionTerritory'].nunique()

### Here we have got that we have the data if around 40 States and Union Territories

## Now let's Analyze the Confirmed,Recovered and Deaths on Each State

In [ ]:
state_wise_cases=state_cases.sort_values('Confirmed', ascending= False).fillna(0).style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Cured"])\
                        .background_gradient(cmap='Oranges',subset=["Active"])\
                        .background_gradient(cmap='RdYlBu',subset=["Death Rate (per 100)"])\
                        .background_gradient(cmap='Accent',subset=["Cure Rate (per 100)"])

In [ ]:
state_wise_cases

Here we can see that Maharastra is at the top of confirmed cases,

# In the Next cell we'll see this in a better visualization 
# Which states are more affected by the virus?

In [ ]:
px.treemap(state_cases,path=["State/UnionTerritory"],values="Deaths",title="Overall States Comparision of deaths")

# Let's Check the Fatality Ratio

In [ ]:
df['Fatality-Ratio'] = df['Deaths']/df['Confirmed']

In [ ]:
df['Fatality-Ratio']

### FATALITY_RATIO AMONG STATES

In [ ]:
fatality_ratio=df.groupby('State/UnionTerritory')['Fatality-Ratio'].sum().reset_index()
fatality_ratio.iplot(kind='line',mode='lines+markers',x='State/UnionTerritory',y='Fatality-Ratio',title='Fatality-Ratio Among States',color='blue')

### FATALITY RATIO IN AN YEAR

In [ ]:
fatality_year=df.groupby('Date')['Fatality-Ratio'].sum().reset_index()
fatality_year.iplot(kind='line',x='Date',y='Fatality-Ratio')

### Top 10 States With Highest Confirmed Cases

In [ ]:
# Top 10 states with maximum number of COnfirmed Cases
top_10=state_cases.groupby('State/UnionTerritory')['Confirmed'].max().sort_values(ascending=False).reset_index()
trace = go.Table(
    domain=dict(x=[0, 0.52],
                y=[0, 1.0]),
    header=dict(values=["State/UnionTerritory","Confirmed Cases"],
                fill = dict(color = '#119DFF'),
                font = dict(color = 'white', size = 14),
                align = ['center'],
               height = 30),
    cells=dict(values=[top_10['State/UnionTerritory'].head(10),top_10['Confirmed'].head(10)],
               fill = dict(color = ['#25FEFD', 'white']),
               align = ['center'],height=20))

trace1 = go.Bar(x=top_10['State/UnionTerritory'].head(10),
                y=top_10['Confirmed'].head(10),
                xaxis='x1',
                yaxis='y1',
                marker=dict(color='blue'),opacity=0.60)
layout = dict(
    width=1000,
    height=400,
    autosize=False,
    title='Top-10 States with Highest No. of Cases',
    showlegend=False,   
    xaxis1=dict(**dict(domain=[0.58, 1], anchor='y1', showticklabels=True)),
    yaxis1=dict(**dict(domain=[0, 1.0], anchor='x1', hoverformat='.2f')),  
)
fig1 = dict(data=[trace, trace1], layout=layout)
iplot(fig1)

### Top 10 States in Highest Recovery

In [ ]:
# Top 10 states with maximum number of Recovery
top_10=state_cases.groupby('State/UnionTerritory')['Cured'].max().sort_values(ascending=False).reset_index()
trace = go.Table(
    domain=dict(x=[0, 0.52],
                y=[0, 1.0]),
    header=dict(values=["State/UnionTerritory","Cured Cases"],
                fill = dict(color = '#119DFF'),
                font = dict(color = 'white', size = 14),
                align = ['center'],
               height = 30),
    cells=dict(values=[top_10['State/UnionTerritory'].head(10),top_10['Cured'].head(10)],
               fill = dict(color = ['lime', 'white']),
               align = ['center'],height=20))

trace1 = go.Bar(x=top_10['State/UnionTerritory'].head(10),
                y=top_10['Cured'].head(10),
                xaxis='x1',
                yaxis='y1',
                marker=dict(color='lime'),opacity=0.60)
layout = dict(
    width=1000,
    height=400,
    autosize=False,
    title='Top-10 States with Highest Recovery',
    showlegend=False,   
    xaxis1=dict(**dict(domain=[0.58, 1], anchor='y1', showticklabels=True)),
    yaxis1=dict(**dict(domain=[0, 1.0], anchor='x1', hoverformat='.2f')),  
)
fig1 = dict(data=[trace, trace1], layout=layout)
iplot(fig1)

### Top 10 States with highest number of Deaths

In [ ]:
# Top 10 states with maximum number of Deaths
top_10=state_cases.groupby('State/UnionTerritory')['Deaths'].max().sort_values(ascending=False).reset_index()
trace = go.Table(
    domain=dict(x=[0, 0.52],
                y=[0, 1.0]),
    header=dict(values=["State/UnionTerritory","Deaths Cases"],
                fill = dict(color = '#119DFF'),
                font = dict(color = 'white', size = 14),
                align = ['center'],
               height = 30),
    cells=dict(values=[top_10['State/UnionTerritory'].head(10),top_10['Deaths'].head(10)],
               fill = dict(color = ['darksalmon', 'white']),
               align = ['center'],height=20))

trace1 = go.Bar(x=top_10['State/UnionTerritory'].head(10),
                y=top_10['Deaths'].head(10),
                xaxis='x1',
                yaxis='y1',
                marker=dict(color='darkred'),opacity=0.70)
layout = dict(
    width=1000,
    height=400,
    autosize=False,
    title='Top-10 States with Highest No. of Cases',
    showlegend=False,   
    xaxis1=dict(**dict(domain=[0.58, 1], anchor='y1', showticklabels=True)),
    yaxis1=dict(**dict(domain=[0, 1.0], anchor='x1', hoverformat='.2f')),  
)
fig1 = dict(data=[trace, trace1], layout=layout)
iplot(fig1)

## Testing Details

In [ ]:
df_testing=pd.read_csv(r"../input/covid19-in-india/StatewiseTestingDetails.csv")

In [ ]:
df_testing.head()

In [ ]:
# States with Total Sample collections
df_sample=df_testing.groupby('State')['TotalSamples'].max().sort_values(ascending=False).reset_index()

In [ ]:
df_sample.head()

In [ ]:
# Top 10 states with Total Sample Collections
px.bar(df_sample.head(10),x='State',y='TotalSamples')

In [ ]:
df_positive=df_testing.groupby('State')['Positive'].max().sort_values(ascending=False).reset_index()

## Positive Cases
## What is the number of positive cases rising in India?

In [ ]:
df_positive.head(10).iplot(kind='bar',x='State',y='Positive')

In [ ]:
df_vaccine=pd.read_csv(r'../input/covid19-in-india/covid_vaccine_statewise.csv')

In [ ]:
df_vaccine.head()

In [ ]:
df_vaccine.columns

In [ ]:
male = df_vaccine["Male(Individuals Vaccinated)"].max() 
female = df_vaccine["Female(Individuals Vaccinated)"].max()  
trans = df_vaccine["Transgender(Individuals Vaccinated)"].max()
px.pie(names=["Male Vaccinated","Female Vaccinated","Trans Gender"],values=[male,female,trans],title="Male and Female Vaccinated ratio for Covid19")

In [ ]:
Covaxin = df_vaccine["Total Covaxin Administered"].max() 
Covishield = df_vaccine["Total CoviShield Administered"].max()  
px.pie(names=["Covaxin Vaccinated","Covishield Vaccinated"],values=[Covaxin,Covishield],title="Covaxin and Covishield Vaccination")

In [ ]:
df_vaccine.head()

In [ ]:
Doses = df_vaccine["Total Doses Administered"].max() 
Vaccinated = df_vaccine["Total Individuals Vaccinated"].max()  
px.pie(names=["Doses Administered","People Vaccinated"],values=[Doses,Vaccinated],title="Doses administered vs People Vaccinated")

In [ ]:
df_vaccine.head()

In [ ]:
df_vaccine['State'].unique()

# Let's Analyze 3 States of How many People gets Vaccinated
# Odisha, Maharastra, Delhi

In [ ]:
df_Odisha=df_vaccine[df_vaccine['State']=='Odisha']
df_Maharashtra=df_vaccine[df_vaccine['State']=='Maharashtra']
df_Delhi=df_vaccine[df_vaccine['State']=='Delhi']

In [ ]:
Doses = df_Odisha["Total Doses Administered"].max() 
Vaccinated = df_Odisha["Total Individuals Vaccinated"].max()  
px.pie(names=["Doses Administered","People Vaccinated"],values=[Doses,Vaccinated],title="Doses administered vs People Vaccinated")

In [ ]:
maxDoses = df_Maharashtra["Total Doses Administered"].max() 
Vaccinated = df_Maharashtra["Total Individuals Vaccinated"].max()  
px.pie(names=["Doses Administered","People Vaccinated"],values=[Doses,Vaccinated],title="Doses administered vs People Vaccinated")

In [ ]:
Doses = df_Delhi["Total Doses Administered"].max() 
Vaccinated = df_Delhi["Total Individuals Vaccinated"].max()  
px.pie(names=["Doses Administered","People Vaccinated"],values=[Doses,Vaccinated],title="Doses administered vs People Vaccinated")

In [ ]:
df_Odisha.iplot(kind='bar',x='Updated On',y='Total Individuals Vaccinated')

# *Let's Predict The Upcoming Cases and Deaths on India*

In [ ]:
df_confirmed_india.columns = ['ds','y']
df_confirmed_india['ds'] = pd.to_datetime(df_confirmed_india['ds'])

In [ ]:
df_confirmed_india

In [ ]:
from fbprophet import Prophet

In [ ]:
m = Prophet()
m.fit(df_confirmed_india)
future = m.make_future_dataframe(periods=21)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast.tail()

In [ ]:
m.plot(forecast)

In [ ]:
confirmed_forecast_plot =m.plot_components(forecast)

# *Let's Predict the Deaths in India*

In [ ]:
df_death_india.columns = ['ds','y']
df_death_india['ds'] = pd.to_datetime(df_death_india['ds'])

In [ ]:
df_death_india

In [ ]:
model = Prophet()
model.fit(df_death_india)
future = model.make_future_dataframe(periods=14)
future.tail()

In [ ]:
prediction = model.predict(future)
prediction.tail()

In [ ]:
prediction.plot()

In [ ]:
death_forecast_plot = model.plot(prediction)

In [ ]:
deaths_forecast_plot = model.plot_components(prediction)

# Thanks a Lot!! Stay Home Stay Safe